In [82]:
# @title ##**required module** { display-mode: "form" }
from IPython.display import clear_output

!pip install opencv-python
!pip install mediapipe
clear_output()

In [83]:
# @title ##**Webcam module** { display-mode: "form" }
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

  
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [84]:
# @title ##**face detection** { display-mode: "form" }

import cv2
import mediapipe as mp 
import time
import numpy as np
class FaceDetector():
    def __init__(self,minDetectionCon = 0.5):
        self.minDetectionCon=minDetectionCon
        self.mpFaceDetection  = mp.solutions.face_detection
        self.mpDraw = mp.solutions.drawing_utils
        self.faceDetection =self.mpFaceDetection.FaceDetection(self.minDetectionCon)
    
    def findFaces(self, img , draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.result = self.faceDetection.process(imgRGB)
        # print(self.result)
        bboxs = []
    

        if self.result.detections:
            for id,detection in enumerate(self.result.detections):
                # mpDraw.draw_detection(img,detection)
                # print(id, detection)
                # print(detection.score)
                # print(detection.location_data.relative_bounding_box)
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, ic =img.shape
                img_blank=np.zeros((ih,iw,4),np.uint8)
                # img_blank=img
                bbox = int(bboxC.xmin *iw),int(bboxC.ymin *ih),\
                    int(bboxC.width *iw), int(bboxC.height *ih)
                bboxs.append([id, bbox, detection.score])
                if draw:
                    img=self.fancyDraw(img_blank,bbox)
                    cv2.putText(img_blank,f'FPS : {int(detection.score[0]*100)}%',(bbox[0],bbox[1]-20),cv2.FONT_HERSHEY_PLAIN,2
                    ,(255,0,255),2)
        return img_blank,bboxs
    def fancyDraw(self, img_blank, bbox , l=30, t=5,rt=1):
        x, y, w, h = bbox
        x1, y1 = x+w , y+h
        cv2.rectangle(img_blank,bbox,(255,0,255),rt)
        
        #Top left x,y
        cv2.line(img_blank, (x,y), (x+l,y),(255,0,255), t )
        cv2.line(img_blank, (x,y), (x,y+l),(255,0,255), t )
        #Top right x1,y
        cv2.line(img_blank, (x1,y), (x1-l,y),(255,0,255), t )
        cv2.line(img_blank, (x1,y), (x1,y+l),(255,0,255), t )
        #Bottom left x,y1
        cv2.line(img_blank, (x,y1), (x+l,y1),(255,0,255), t )
        cv2.line(img_blank, (x,y1), (x,y1-l),(255,0,255), t )
        #Bottom right x1,y
        cv2.line(img_blank, (x1,y1), (x1-l,y1),(255,0,255), t )
        cv2.line(img_blank, (x1,y1), (x1,y1-l),(255,0,255), t )

        return img_blank


# def main():
#     # cap=cv2.VideoCapture("videos/2.mp4")
#     cap=cv2.VideoCapture(0)
#     pTime = 0

#     detector= FaceDetector()
#     while True:
#         success, img =cap.read()
#         img=cv2.flip(img,1)
#         img, bboxs = detector.findFaces(img,True)
#         print(bboxs)
#         cTime = time.time()
#         fps = 1/(cTime -pTime)
#         pTime =cTime
#         cv2.putText(img,f'FPS : {int(fps)}',(20,70),cv2.FONT_HERSHEY_PLAIN,3
#         ,(0,255,0),2)
#         cv2.imshow("Image",img)
#         cv2.waitKey(1)
# if __name__ == "__main__":
#     main()    

In [87]:
# @title ##**Run this to start realtime face detection** { display-mode: "form" }

def main():
    video_stream()
    label_html = 'Capturing...'
    bbox = ''
    count = 0 
    pTime = 0

    detector= FaceDetector()
    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break
        img = js_to_image(js_reply["img"])
        # img=cv2.flip(img,1)
        try:
          img_blank, bboxs = detector.findFaces(img,True)
          img_blank[:,:,3] = (img_blank.max(axis = 2) > 0 ).astype(int) * 255
          bbox_bytes = bbox_to_bytes(img_blank)
          bbox = bbox_bytes
        except:
          ih, iw, ic =img.shape
          img_blank=np.zeros((ih,iw,4),np.uint8)
          img_blank[:,:,3] = (img_blank.max(axis = 2) > 0 ).astype(int) * 255
          bbox_bytes = bbox_to_bytes(img_blank)
          bbox = bbox_bytes

if __name__ == "__main__":
    main()    

<IPython.core.display.Javascript object>